In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import warnings

warnings.filterwarnings('ignore')

train_path = 'C:/Users/f3107/Desktop/hy_data/hy_round1_train_20200102'
test_path = 'C:/Users/f3107/Desktop/hy_data/hy_round1_testA_20200102'

### 文件夹基本信息

In [2]:
train_files = os.listdir(train_path)
test_files = os.listdir(test_path)
print(len(train_files), len(test_files))

7000 2000


In [3]:
train_files[:3]

['0.csv', '1.csv', '10.csv']

In [4]:
test_files[:3]

['7000.csv', '7001.csv', '7002.csv']

### 单个文件内容

In [5]:
df = pd.read_csv(f'{train_path}/6966.csv')

In [6]:
df.head()

,渔船ID,x,y,速度,方向,time,type
0,6966,6.265902e+06,5.279254e+06,0.11,306,1106 23:58:16,围网
1,6966,6.265902e+06,5.279254e+06,0.00,0,1106 23:48:21,围网
2,6966,6.265902e+06,5.279254e+06,0.00,0,1106 23:38:19,围网
3,6966,6.265902e+06,5.279254e+06,0.00,0,1106 23:28:36,围网
4,6966,6.265902e+06,5.279254e+06,0.32,130,1106 23:08:17,围网


### 整合单个文件

#### train

In [7]:
#未处理，时间倒序
ret = []
for file in tqdm(train_files):
    df = pd.read_csv(f'{train_path}/{file}')
    ret.append(df)
df = pd.concat(ret)
df.columns = ['ship_ID','x','y','speed_v','direction','datetime','type']

100%|█████████████████████████████████████████████████████████████████████████████| 7000/7000 [00:15<00:00, 455.27it/s]


In [8]:
df.to_hdf('C:/Users/f3107/Desktop/hy_data/train.h5', 'df', mode='w')

In [9]:
#已处理，时间顺序
ret = []
for file in tqdm(train_files):
    df = pd.read_csv(f'{train_path}/{file}')
    df = df.sort_index(ascending=False)
    ret.append(df)
df = pd.concat(ret)
df.columns = ['ship_ID','x','y','speed_v','direction','datetime','type']

100%|█████████████████████████████████████████████████████████████████████████████| 7000/7000 [00:18<00:00, 387.50it/s]


In [10]:
df.to_hdf('C:/Users/f3107/Desktop/hy_data/train_descending.h5', 'df', mode='w')

#### test

In [11]:
#未处理，时间倒序
ret = []
for file in tqdm(test_files):
    df = pd.read_csv(f'{test_path}/{file}')
    ret.append(df)
df = pd.concat(ret)
df.columns = ['ship_ID','x','y','speed_v','direction','datetime','type']

100%|█████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:04<00:00, 483.42it/s]


ValueError: Length mismatch: Expected axis has 6 elements, new values have 7 elements

In [ ]:
df.to_hdf('C:/Users/f3107/Desktop/hy_data/test.h5', 'df', mode='w')

In [ ]:
#已处理，时间顺序
ret = []
for file in tqdm(test_files):
    df = pd.read_csv(f'{test_path}/{file}')
    df = df.sort_index(ascending=False)
    ret.append(df)
df = pd.concat(ret)
df.columns = ['ship_ID','x','y','speed_v','direction','datetime','type']

In [ ]:
df.to_hdf('C:/Users/f3107/Desktop/hy_data/test_descending.h5', 'df', mode='w')